# Tile Coding
---

Tile coding is an innovative way of discretizing a continuous space that enables better generalization compared to a single grid-based approach. The fundamental idea is to create several overlapping grids or _tilings_; then for any given sample value, you need only check which tiles it lies in. You can then encode the original continuous value by a vector of integer indices or bits that identifies each activated tile.

### 1. Import the Necessary Packages

In [ ]:
# Import common libraries
import pickle
import sys
import gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Set plotting options
%matplotlib inline
plt.style.use('ggplot')
np.set_printoptions(precision=3, linewidth=120)

### 2. Specify the Environment, and Explore the State and Action Spaces

We'll use [OpenAI Gym](https://gym.openai.com/) environments to test and develop our algorithms. These simulate a variety of classic as well as contemporary reinforcement learning tasks.  Let's begin with an environment that has a continuous state space, but a discrete action space.

Link to [Acrobot sources](https://github.com/openai/gym/blob/master/gym/envs/classic_control/acrobot.py) for a better explanations of states and actions.

In [ ]:
# Create an environment
env = gym.make('Acrobot-v1')
env.seed(505);

# Explore state (observation) space
print("State space:", env.observation_space)
print("- low:", env.observation_space.low)
print("- high:", env.observation_space.high)

# Explore action space
print("Action space:", env.action_space)

Note that the state space is multi-dimensional, with most dimensions ranging from -1 to 1 (positions of the two joints), while the final two dimensions have a larger range. How do we discretize such a space using tiles?

### 3. Tiling

Let's first design a way to create a single tiling for a given state space. This is very similar to a uniform grid! The only difference is that you should include an offset for each dimension that shifts the split points.

For instance, if `low = [-1.0, -5.0]`, `high = [1.0, 5.0]`, `bins = (10, 10)`, and `offsets = (-0.1, 0.5)`, then return a list of 2 NumPy arrays (2 dimensions) each containing the following split points (9 split points per dimension):

```
[array([-0.9, -0.7, -0.5, -0.3, -0.1,  0.1,  0.3,  0.5,  0.7]),
 array([-3.5, -2.5, -1.5, -0.5,  0.5,  1.5,  2.5,  3.5,  4.5])]
```

Notice how the split points for the first dimension are offset by `-0.1`, and for the second dimension are offset by `+0.5`. This might mean that some of our tiles, especially along the perimeter, are partially outside the valid state space, but that is unavoidable and harmless.

In [ ]:
def create_tiling_grid(low, high, bins=(10, 10), offsets=(0.0, 0.0)):
    """Define a uniformly-spaced grid that can be used for tile-coding a space.
    
    Parameters
    ----------
    low : array_like
        Lower bounds for each dimension of the continuous space.
    high : array_like
        Upper bounds for each dimension of the continuous space.
    bins : tuple
        Number of bins or tiles along each corresponding dimension.
    offsets : tuple
        Split points for each dimension should be offset by these values.
    
    Returns
    -------
    grid : list of array_like
        A list of arrays containing split points for each dimension.
    """
    # TODO: Implement this
    ranges = np.array(high) - np.array(low)
    # print(ranges)
    bins_sizes = ranges / bins
    # print(bins_sizes)
    grid = []
    for bin_dim, bin_num in enumerate(bins):
        split_dim = np.linspace(
                        start=low[bin_dim] + bins_sizes[bin_dim] + offsets[bin_dim],
                        stop=high[bin_dim] + offsets[bin_dim],
                        num=bins[bin_dim] - 1,
                        endpoint=False
                    )
        grid.append(split_dim)
    #print(grid)
    return grid


low = [-1.0, -5.0]
high = [1.0, 5.0]
create_tiling_grid(low, high, bins=(10, 10), offsets=(-0.1, 0.5))  # [test]

You can now use this function to define a set of tilings that are a little offset from each other.

In [ ]:
def create_tilings(low, high, tiling_specs):
    """Define multiple tilings using the provided specifications.

    Parameters
    ----------
    low : array_like
        Lower bounds for each dimension of the continuous space.
    high : array_like
        Upper bounds for each dimension of the continuous space.
    tiling_specs : list of tuples
        A sequence of (bins, offsets) to be passed to create_tiling_grid().

    Returns
    -------
    tilings : list
        A list of tilings (grids), each produced by create_tiling_grid().
    """
    # TODO: Implement this
    return [create_tiling_grid(low, high, bins=tiling_spec[0], offsets=tiling_spec[1]) for tiling_spec in tiling_specs]


# Tiling specs: [(<bins>, <offsets>), ...]
tiling_specs = [((10, 10), (-0.066, -0.33)),
                ((10, 10), (0.0, 0.0)),
                ((10, 10), (0.066, 0.33))]
tilings = create_tilings(low, high, tiling_specs)
tilings

It may be hard to gauge whether you are getting desired results or not. So let's try to visualize these tilings.

In [ ]:
from matplotlib.lines import Line2D

def visualize_tilings(tilings):
    """Plot each tiling as a grid."""
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    linestyles = ['-', '--', ':']
    legend_lines = []

    fig, ax = plt.subplots(figsize=(10, 10))
    for i, grid in enumerate(tilings):
        for x in grid[0]:
            l = ax.axvline(x=x, color=colors[i % len(colors)], linestyle=linestyles[i % len(linestyles)], label=i)
        for y in grid[1]:
            l = ax.axhline(y=y, color=colors[i % len(colors)], linestyle=linestyles[i % len(linestyles)])
        legend_lines.append(l)
    ax.grid('off')
    ax.legend(legend_lines, ["Tiling #{}".format(t) for t in range(len(legend_lines))], facecolor='white', framealpha=0.9)
    ax.set_title("Tilings")
    return ax  # return Axis object to draw on later, if needed


visualize_tilings(tilings);

Great! Now that we have a way to generate these tilings, we can next write our encoding function that will convert any given continuous state value to a discrete vector.

### 4. Tile Encoding

Implement the following to produce a vector that contains the indices for each tile that the input state value belongs to. The shape of the vector can be the same as the arrangment of tiles you have, or it can be ultimately flattened for convenience.

You can use the same `discretize()` function here from grid-based discretization, and simply call it for each tiling.

In [ ]:
def discretize(sample, grid):
    """Discretize a sample as per given grid.
    
    Parameters
    ----------
    sample : array_like
        A single sample from the (original) continuous space.
    grid : list of array_like
        A list of arrays containing split points for each dimension.
    
    Returns
    -------
    discretized_sample : array_like
        A sequence of integers with the same number of dimensions as sample.
    """
    # TODO: Implement this
    assert len(sample) == len(grid)
    # print(sample)
    bin_indexes = [np.digitize(sample_dim, grid_dim) for sample_dim, grid_dim in zip(sample, grid)]
    # print(bin_indexes)
    return bin_indexes


def tile_encode(sample, tilings, flatten=False):
    """Encode given sample using tile-coding.
    
    Parameters
    ----------
    sample : array_like
        A single sample from the (original) continuous space.
    tilings : list
        A list of tilings (grids), each produced by create_tiling_grid().
    flatten : bool
        If true, flatten the resulting binary arrays into a single long vector.

    Returns
    -------
    encoded_sample : list or array_like
        A list of binary vectors, one for each tiling, or flattened into one.
    """
    # TODO: Implement this
    sample_encodings = [discretize(sample, tiling) for tiling in tilings]
    # print(sample_encodings)
    if flatten:
        flatten_encodings = []
        for sample_encoding in sample_encodings:
            flatten_encodings.extend(sample_encoding)
        return flatten_encodings
    return sample_encodings


# Test with some sample values
samples = [(-1.2 , -5.1 ),
           (-0.75,  3.25),
           (-0.5 ,  0.0 ),
           ( 0.25, -1.9 ),
           ( 0.15, -1.75),
           ( 0.75,  2.5 ),
           ( 0.7 , -3.7 ),
           ( 1.0 ,  5.0 )]
encoded_samples = [tile_encode(sample, tilings, flatten=False) for sample in samples]
print("\nSamples:", repr(samples), sep="\n")
print("\nEncoded samples:", repr(encoded_samples), sep="\n")

Note that we did not flatten the encoding above, which is why each sample's representation is a pair of indices for each tiling. This makes it easy to visualize it using the tilings.

In [ ]:
from matplotlib.patches import Rectangle

def visualize_encoded_samples(samples, encoded_samples, tilings, low=None, high=None):
    """Visualize samples by activating the respective tiles."""
    samples = np.array(samples)  # for ease of indexing

    # Show tiling grids
    ax = visualize_tilings(tilings)
    
    # If bounds (low, high) are specified, use them to set axis limits
    if low is not None and high is not None:
        ax.set_xlim(low[0], high[0])
        ax.set_ylim(low[1], high[1])
    else:
        # Pre-render (invisible) samples to automatically set reasonable axis limits, and use them as (low, high)
        ax.plot(samples[:, 0], samples[:, 1], 'o', alpha=0.0)
        low = [ax.get_xlim()[0], ax.get_ylim()[0]]
        high = [ax.get_xlim()[1], ax.get_ylim()[1]]

    # Map each encoded sample (which is really a list of indices) to the corresponding tiles it belongs to
    tilings_extended = [np.hstack((np.array([low]).T, grid, np.array([high]).T)) for grid in tilings]  # add low and high ends
    tile_centers = [(grid_extended[:, 1:] + grid_extended[:, :-1]) / 2 for grid_extended in tilings_extended]  # compute center of each tile
    tile_toplefts = [grid_extended[:, :-1] for grid_extended in tilings_extended]  # compute topleft of each tile
    tile_bottomrights = [grid_extended[:, 1:] for grid_extended in tilings_extended]  # compute bottomright of each tile

    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    for sample, encoded_sample in zip(samples, encoded_samples):
        for i, tile in enumerate(encoded_sample):
            # Shade the entire tile with a rectangle
            topleft = tile_toplefts[i][0][tile[0]], tile_toplefts[i][1][tile[1]]
            bottomright = tile_bottomrights[i][0][tile[0]], tile_bottomrights[i][1][tile[1]]
            ax.add_patch(Rectangle(topleft, bottomright[0] - topleft[0], bottomright[1] - topleft[1],
                                   color=colors[i], alpha=0.33))

            # In case sample is outside tile bounds, it may not have been highlighted properly
            if any(sample < topleft) or any(sample > bottomright):
                # So plot a point in the center of the tile and draw a connecting line
                cx, cy = tile_centers[i][0][tile[0]], tile_centers[i][1][tile[1]]
                ax.add_line(Line2D([sample[0], cx], [sample[1], cy], color=colors[i]))
                ax.plot(cx, cy, 's', color=colors[i])
    
    # Finally, plot original samples
    ax.plot(samples[:, 0], samples[:, 1], 'o', color='r')

    ax.margins(x=0, y=0)  # remove unnecessary margins
    ax.set_title("Tile-encoded samples")
    return ax

visualize_encoded_samples(samples, encoded_samples, tilings);

Inspect the results and make sure you understand how the corresponding tiles are being chosen. Note that some samples may have one or more tiles in common.

### 5. Q-Table with Tile Coding

The next step is to design a special Q-table that is able to utilize this tile coding scheme. It should have the same kind of interface as a regular table, i.e. given a `<state, action>` pair, it should return a `<value>`. Similarly, it should also allow you to update the `<value>` for a given `<state, action>` pair (note that this should update all the tiles that `<state>` belongs to).

The `<state>` supplied here is assumed to be from the original continuous state space, and `<action>` is discrete (and integer index). The Q-table should internally convert the `<state>` to its tile-coded representation when required.

In [ ]:
class QTable:
    """Simple Q-table."""

    def __init__(self, state_size, action_size):
        """Initialize Q-table.
        
        Parameters
        ----------
        state_size : tuple
            Number of discrete values along each dimension of state space.
        action_size : int
            Number of discrete actions in action space.
        """
        self.state_size = state_size
        self.action_size = action_size

        # print(state_size)
        # print(action_size)
        # TODO: Create Q-table, initialize all Q-values to zero
        # Note: If state_size = (9, 9), action_size = 2, q_table.shape should be (9, 9, 2)
        self.q_table = np.zeros(tuple(state_size) + (action_size,), dtype=np.float32)
        print("QTable(): size =", self.q_table.shape)


class TiledQTable:
    """Composite Q-table with an internal tile coding scheme."""
    
    def __init__(self, low, high, tiling_specs, action_size):
        """Create tilings and initialize internal Q-table(s).
        
        Parameters
        ----------
        low : array_like
            Lower bounds for each dimension of state space.
        high : array_like
            Upper bounds for each dimension of state space.
        tiling_specs : list of tuples
            A sequence of (bins, offsets) to be passed to create_tilings() along with low, high.
        action_size : int
            Number of discrete actions in action space.
        """
        self.tilings = create_tilings(low, high, tiling_specs)
        # print(self.tilings)
        self.state_sizes = [tuple(len(splits)+1 for splits in tiling_grid) for tiling_grid in self.tilings]
        # print(self.state_sizes)
        self.action_size = action_size
        self.q_tables = [QTable(state_size, self.action_size) for state_size in self.state_sizes]
        print("TiledQTable(): no. of internal tables = ", len(self.q_tables))
    
    def get_state_action(self, state, action):
        """Get Q-value for given <state, action> pair.
        
        Parameters
        ----------
        state : array_like
            Vector representing the state in the original continuous space.
        action : int
            Index of desired action.
        
        Returns
        -------
        value : float
            Q-value of given <state, action> pair, averaged from all internal Q-tables.
        """
        # TODO: Encode state to get tile indices
        # print(state)
        encodings_state = tile_encode(state, self.tilings)
        # print(encodings_state)
        # TODO: Retrieve q-value for each tiling, and return their average
        q_values = [self.q_tables[tile_ix].q_table[tuple(encoding_state) + (action,)] \
                    for tile_ix, encoding_state in enumerate(encodings_state)]
        return np.mean(q_values)
    
    def get_state(self, state):
        """Get Q-values for given <state> pair.
        
        Parameters
        ----------
        state : array_like
            Vector representing the state in the original continuous space.
        
        Returns
        -------
        value : array_like
            Q-values of given state from all internal Q-tables.
        """
        # TODO: Encode state to get tile indices
        # print(state)
        encodings_state = tile_encode(state, self.tilings)
        # print(encodings_state)
        # TODO: Retrieve q-value for each tiling, and return their average
        q_values = [self.q_tables[tile_ix].q_table[tuple(encoding_state) + (action,)] \
                    for tile_ix, encoding_state in enumerate(encodings_state)]
        return q_values
    
    def update(self, state, action, value, alpha=0.1):
        """Soft-update Q-value for given <state, action> pair to value.
        
        Instead of overwriting Q(state, action) with value, perform soft-update:
            Q(state, action) = alpha * value + (1.0 - alpha) * Q(state, action)
        
        Parameters
        ----------
        state : array_like
            Vector representing the state in the original continuous space.
        action : int
            Index of desired action.
        value : float
            Desired Q-value for <state, action> pair.
        alpha : float
            Update factor to perform soft-update, in [0.0, 1.0] range.
        """
        # TODO: Encode state to get tile indices
        encodings_state = tile_encode(state, self.tilings)
        # TODO: Update q-value for each tiling by update factor alpha
        for tile_ix, q_table in enumerate(self.q_tables):
            old_value = q_table.q_table[tuple(encodings_state[tile_ix]) + (action,)]
            q_table.q_table[tuple(encodings_state[tile_ix]) + (action,)] = \
                alpha * value + (1.0 - alpha) * old_value


# Test with a sample Q-table
tq = TiledQTable(low, high, tiling_specs, action_size=2)
s1 = 3; s2 = 4; a = 0; q = 1.0
print("[GET]    Q({}, {}) = {}".format(samples[s1], a, tq.get_state_action(samples[s1], a)))  # check value at sample = s1, action = a
print("[UPDATE] Q({}, {}) = {}".format(samples[s2], a, q)); tq.update(samples[s2], a, q)  # update value for sample with some common tile(s)
print("[GET]    Q({}, {}) = {}".format(samples[s1], a, tq.get_state_action(samples[s1], a)))  # check value again, should be slightly updated

If you update the q-value for a particular state (say, `(0.25, -1.91)`) and action (say, `0`), then you should notice the q-value of a nearby state (e.g. `(0.15, -1.75)` and same action) has changed as well! This is how tile-coding is able to generalize values across the state space better than a single uniform grid.

### 6. Implement a Q-Learning Agent using Tile-Coding

Now it's your turn to apply this discretization technique to design and test a complete learning agent! 

In [ ]:
# Create an environment
env = gym.make('Acrobot-v1')

env_low = env.observation_space.low
env_high = env.observation_space.high

# Explore state (observation) space
print("State space:", env.observation_space)
print("- low:", env_low)
print("- high:", env_high)

# Explore action space
print("Action space:", env.action_space)

Random experiment

In [ ]:
state = env.reset()
score = 0
for t in range(200):
    action = env.action_space.sample()
    env.render()
    state, reward, done, _ = env.step(action)
    score += reward
    if done:
        break 
print('Final score:', score)
env.close()


QLearningAgent Class

In [ ]:
class QLearningAgent:
    """Q-Learning agent that can act on a continuous state space by discretizing it."""

    def __init__(self, env, tiled_qtable, alpha=0.02, gamma=0.99,
                 start_epsilon=1.0, epsilon_decay_rate=0.99, min_epsilon=0.01):
        self.env = env
        self.action_size = self.env.action_space.n
        self.tiled_qtable = tiled_qtable
        self.alpha = alpha
        # discount factor
        self.gamma = gamma
        # epsilon
        self.epsilon = start_epsilon
        self.epsilon_decay_rate = epsilon_decay_rate
        self.min_epsilon = min_epsilon
        # Last state for every q_table
        self.last_state = None
        # Last state (encoded) for every q_table
        self.last_encodings_state = None
        # Last taken action
        self.last_action = None
    
    def get_best_action(self, encodings_state):
        # VOTING
        best_actions = []
        for tile_ix, encoding_state in enumerate(encodings_state):
            # print(self.tiled_qtable.q_tables[tile_ix].q_table[tuple(encoding_state)])
            tile_best_action = np.argmax(self.tiled_qtable.q_tables[tile_ix].q_table[tuple(encoding_state)])
            best_actions.append(tile_best_action)            
        # Vote the most frequent action
        return np.bincount(best_actions).argmax()
    
        # AVG + MAX
        # return np.argmax(self.tiled_qtable.get(state, action)
    
    def get_q_value(self, state, action):
        return self.tiled_qtable.get_state_action(state, action)
    
    def reset_episode(self, state):
        # Update epsilon
        self.epsilon *= self.epsilon_decay_rate
        self.epsilon = max(self.epsilon, self.min_epsilon)
        # Reset last_state and last_action
        self.last_state = state
        self.last_encodings_state = tile_encode(state, self.tiled_qtable.tilings)
        # Caveat: this action is not really taken, we start with the best action for initial state
        self.last_action = self.get_best_action(self.last_encodings_state)
        return self.last_action
    
    def get_action(self, state, reward, mode="train"):
        assert self.last_state is not None
        assert self.last_action is not None
        # Get encodings for every tiling
        encodings_state = tile_encode(state, self.tiled_qtable.tilings)
        # Get best action for every tiling
        # print(encodings_state)
        if mode == "test":
            return self.get_best_action(encodings_state)
        elif mode == "train":
            # TODO: Learn Q considering reward for last_state and last_action
            
            # New component for Q: reward + gamma * max_a(Q(S_t+1, a))
            # state == S_t+1
            # last_state == S_t
            # last_action == a_t
            # take_action == a_t+1
                
            # Get avg Q value
            new_value = reward + self.gamma * max(self.tiled_qtable.get_state(state=state))  
            # Q(S_t, a_t) = alpha * new_value + (1 - alpha) * old_value
            
            # Update all the tiles
            self.tiled_qtable.update(
                state=self.last_state,
                action=self.last_action,
                value=new_value,
                alpha=self.alpha
            )
            
            if np.random.uniform(0, 1) < self.epsilon:
                # Exploration -> select a random action among random actions (for every tile)
                take_action = np.random.randint(0, self.action_size)
            else:
                # Exploitation -> select the most voted action
                take_action = self.get_best_action(encodings_state)
            
            self.last_action = take_action
            self.last_state = state
            self.last_encodings_state = encodings_state
            return take_action
        else:
            raise Exception(f"Mode {mode} unknown")

Train and test functions

In [ ]:
def run(env, agent, num_episodes=10000, test_steps=50, mode="train", test_with_gui=False):
    print("Running...")
    scores = []
    
    for num_episode in range(num_episodes):
        init_state = env.reset()
        done = False
        # We take always the exploitation action as first action,
        # to start with a consistent last_action, last_state and reward triplet
        # The reward depends on init_state + first_action and we end at state
        action = agent.reset_episode(init_state)
        state, reward, done, info = env.step(action)
        score = reward
        while not done:
            action = agent.get_action(state, reward=reward, mode=mode)
            state, reward, done, info = env.step(action)
            score += reward
        
        # The score is the cumulative reward
        scores.append(score)
        
        if (num_episode + 1) % test_steps == 0:
            if mode == "train":
                with open(f"agent_{num_episode + 1}.pkl", "wb") as file_out:
                    pickle.dump(agent, file_out)
            print(f"Num episode {num_episode + 1} / {num_episodes}, " + \
                  f"Avg score last {test_steps} runs: {np.mean(scores[-test_steps:])}")
            print(f"Reached epsilon: {agent.epsilon}")
            if test_with_gui:
                run_test_episode_with_gui(env, agent)

In [ ]:
def run_test_episode_with_gui(env, agent):
    state = env.reset()
    env.render()
    done = False
    score = 0
    while not done:
        action = agent.get_action(state, 0.0, "test")
        state, reward, done, info = env.step(action)
        env.render()
        score += reward
    print(f"Episode score: {score}")
    env.close()

Agent initialization

In [ ]:
bins = (10, 10, 10, 10, 20, 20)
start_epsilon = 1.0  # Probability to do exploration instead of exploitation.
epsilon_decay_rate = 0.998 # E.g. 0.995^800 * 1.0 = 0.018, so about 800-900 episodes to get the minimum epsilon 
min_epsilon = 0.01
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount rate
num_episodes = 20000
env_tiling_specs = [
    (bins, (-0.1, -0.1, -0.1, -0.1, -1.0, -1.0)),
    (bins, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)),
    (bins, (0.1, 0.1, 0.1, 0.1, 1.0, 1.0))
]
env_tqt = TiledQTable(
    low=env_low,
    high=env_high,
    tiling_specs=env_tiling_specs,
    action_size=int(env.action_space.n))
agent = QLearningAgent(
    env,
    env_tqt,
    alpha=alpha,
    start_epsilon=start_epsilon,
    epsilon_decay_rate=epsilon_decay_rate,
    min_epsilon=min_epsilon,
    gamma=gamma
)

Learn

In [ ]:
run(env, agent, num_episodes, test_steps=100, mode="train", test_with_gui=True)

Try the trained agent

In [ ]:
# Load a model from pkl if you want to reuse an old model
run_test_episode_with_gui(env, agent)

Test the agent (set num_episodes == test_steps to get the average score over all episodes)

In [ ]:
run(env, agent, num_episodes=1000, test_steps=1000, mode="test")